# Section 602 - CONSTRUCTION CLASSIFICATION

## Building Code

### TABLE 602 FIRE-RESISTANCE RATING REQUIREMENTS FOR EXTERIOR WALLS BASED ON FIRE SEPARATION DISTANCEa, d, g 

In [38]:
import json 
# Load the table from the JSON file
with open('table_602.json', 'r') as f:
    table_602 = json.load(f)

In [39]:
#test
def get_fire_resistance_value(data, range_value, construction_type, occupancy):
    for item in data["FIRE_SEPARATION_DISTANCE"]:
        if item["range"] == range_value and item["TYPE_OF_CONSTRUCTION"] == construction_type:
            return item["occupancies"].get(occupancy, "Occupancy group not found")

# Example usage
range_value = "< 5"
construction_type = "All"
occupancy = "F-1"

result = get_fire_resistance_value(table_602, range_value, construction_type, occupancy)
#print("Fire resistance rating:", result)

In [40]:
import sys
import os

# Add the path to the Rule Notebooks folder to the system path
sys.path.append(os.path.abspath('../..'))
# Import the find_value function from table_lookup.py
from ruleFunctions import find_value, check_value, get_building_info, create_scrollable_table

In [41]:
# Define the SPARQL endpoint URL and query file path
endpoint_url = "http://localhost:3030/SmartReviewHotel/sparql"

# Call the function to get the building information
main_construction_type, subtype, occupancy_group, sprinkler_system, sprinkler_type, storey_num = get_building_info(endpoint_url)
#print(main_construction_type, subtype, occupancy_group, sprinkler_system, sprinkler_type, storey_num)

In [42]:
def get_range(distance):
    if distance < 5:
        return "< 5"
    elif 5 <= distance < 10:
        return "5 <= X < 10"
    elif 10 <= distance < 30:
        return "10 <= X < 30"
    else:
        return "<= 30"

def adjust_construction_type(distance, main_construction_type):
    # Adjust the construction type based on the distance and provided type
    adjusted_construction_type = main_construction_type
    if distance < 5 or distance > 30:
        adjusted_construction_type = "All"
    elif 5 <= distance < 10 and main_construction_type not in ["IA"]:
        adjusted_construction_type = "Others"
    elif 10 <= distance < 30 and main_construction_type not in ["IA", "IB", "IIB", "VB"]:
        adjusted_construction_type = "Others"
    return adjusted_construction_type

def get_value_from_lookup(distance, construction_type, occupancy_group):
    range_value = get_range(distance)
    for entry in table_602["FIRE_SEPARATION_DISTANCE"]:
        if entry["range"] == range_value and entry["TYPE_OF_CONSTRUCTION"] == construction_type:
            return entry.get(f"OCCUPANCY_GROUP_{occupancy_group}", None)
    return None

def combine_construction_type(mainConstructionType, subtype):
    # Remove "Type" and any extra whitespace from mainConstructionType
    mainType = mainConstructionType.replace("Type", "").strip()
    # Combine the modified mainType with the subtype
    combinedType = mainType + subtype
    return combinedType


In [43]:
#print(results)

In [44]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper(endpoint_url)
with open("..\..\..\SPARQL\exteriorNonbearing.sparql", "r") as file:
    sparql_query = file.read()

# Set the SPARQL query and the return format
sparql.setQuery(sparql_query)
sparql.setReturnFormat(JSON)

# Execute the query and get the results in JSON format
results = sparql.query().convert()

In [45]:
#print(results)

### Exterior Nonbearing Walls
The minimum required fire resistive rating in hours for Exterior Nonbearing Walls varies based on the occupancy and the Fire Separation Distance associated with the wall. For walls that are associated with multiple occupancies, the most restrictive fire resistance rating was used.All Exterior Nonbearing Walls in the building have met or exceeded this minimum required value. Please see the below table for a detailed analysis.

In [46]:
import pandas as pd
from IPython.display import display, HTML

# Result table processing
result_table = []

# Process each result in the 'bindings' list
for row in results["results"]["bindings"]:
    wall_id = row["batid"]["value"]
    fire_rating = float(row["fireRating"]["value"])
    distance = float(row["smrvExteriorFireSeparationDistanceValue"]["value"])

    # Step 1: Get the range
    newRange = get_range(distance)

    # Step 2: Get the combined construction type
    main_construction_type = "Type V"  # Example value
    subtype = "A"  # Example value
    combinedConstruction = combine_construction_type(main_construction_type, subtype)

    # Step 3: Adjust the construction type based on range
    adjustedConstructionType = adjust_construction_type(distance, combinedConstruction)

    # Step 4: Lookup fire resistance requirement and ensure it's a float
    required_fire_resistance = float(get_fire_resistance_value(table_602, newRange, adjustedConstructionType, occupancy_group))

    # Step 5: Determine Pass/Fail
    pass_fail = "Pass" if fire_rating >= required_fire_resistance else "Fail"

    # Step 6: Construct reasoning
    reasoning = (
        f"The exterior nonbearing wall is ({distance} ft) from the property line. "
        f"The distance to property line requires that the wall have a fire rating of at least "
        f"({required_fire_resistance} hours). The wall has a fire rating of ({fire_rating} hours) "
        f"which {'meets or exceeds' if pass_fail == 'Pass' else 'does not meet'} the minimum required rating."
    )

    # Append to result table
    result_table.append({
        "Wall ID": wall_id,
        "Result": pass_fail,
        "Reasoning": reasoning
    })

# Convert result table into a Pandas DataFrame
df = pd.DataFrame(result_table)

# Make the DataFrame scrollable in Jupyter
display(df.style.set_table_attributes("style='display:inline-block; max-height:300px; overflow-y:scroll;'"))



,Wall ID,Result,Reasoning
0,364234,Pass,The exterior nonbearing wall is (100.71815891860547 ft) from the property line. The distance to property line requires that the wall have a fire rating of at least (0.0 hours). The wall has a fire rating of (1.0 hours) which meets or exceeds the minimum required rating.
1,361109,Pass,The exterior nonbearing wall is (70.83797663989684 ft) from the property line. The distance to property line requires that the wall have a fire rating of at least (0.0 hours). The wall has a fire rating of (1.0 hours) which meets or exceeds the minimum required rating.
2,361108,Pass,The exterior nonbearing wall is (31.943810227181775 ft) from the property line. The distance to property line requires that the wall have a fire rating of at least (0.0 hours). The wall has a fire rating of (1.0 hours) which meets or exceeds the minimum required rating.
3,361111,Pass,The exterior nonbearing wall is (91.4364908821414 ft) from the property line. The distance to property line requires that the wall have a fire rating of at least (0.0 hours). The wall has a fire rating of (1.0 hours) which meets or exceeds the minimum required rating.
4,361110,Pass,The exterior nonbearing wall is (352.85824283462665 ft) from the property line. The distance to property line requires that the wall have a fire rating of at least (0.0 hours). The wall has a fire rating of (1.0 hours) which meets or exceeds the minimum required rating.
5,361119,Pass,The exterior nonbearing wall is (136.21625048890024 ft) from the property line. The distance to property line requires that the wall have a fire rating of at least (0.0 hours). The wall has a fire rating of (1.0 hours) which meets or exceeds the minimum required rating.
6,361118,Pass,The exterior nonbearing wall is (208.91207634303717 ft) from the property line. The distance to property line requires that the wall have a fire rating of at least (0.0 hours). The wall has a fire rating of (1.0 hours) which meets or exceeds the minimum required rating.
7,361113,Pass,The exterior nonbearing wall is (93.91596528318124 ft) from the property line. The distance to property line requires that the wall have a fire rating of at least (0.0 hours). The wall has a fire rating of (1.0 hours) which meets or exceeds the minimum required rating.
8,361112,Pass,The exterior nonbearing wall is (91.9382703059302 ft) from the property line. The distance to property line requires that the wall have a fire rating of at least (0.0 hours). The wall has a fire rating of (1.0 hours) which meets or exceeds the minimum required rating.
9,361114,Pass,The exterior nonbearing wall is (129.9630179393401 ft) from the property line. The distance to property line requires that the wall have a fire rating of at least (0.0 hours). The wall has a fire rating of (1.0 hours) which meets or exceeds the minimum required rating.
